In [1]:
from py4j.java_gateway import JavaGateway, GatewayParameters
# gateway = JavaGateway(gateway_parameters=GatewayParameters(auto_field=True))    
gateway = JavaGateway(gateway_parameters=GatewayParameters(address='localhost', port=25333))
analyst=gateway.entry_point.getTimeAnalyst()
print(type(analyst))
print(type(gateway))

<class 'py4j.java_gateway.JavaObject'>
<class 'py4j.java_gateway.JavaGateway'>


In [2]:
normalizer=analyst.getNormalizer()
normalizer.parse("Hi，all.下周一下午三点开会")
units = normalizer.getTimeUnit()
for unit in units:
    print(unit.getTime())

Mon Jul 08 15:00:00 GMT 2019


In [5]:
gateway.help(units)

Help on class TimeUnit[] in package com.time.nlp:

TimeUnit[] implements java.lang.Cloneable, java.io.Serializable {
|  
|  Methods defined here:
|  
|  ------------------------------------------------------------
|  Fields defined here:
|  
|  ------------------------------------------------------------
|  Internal classes defined here:
|  
}


In [7]:
DateUtil=gateway.jvm.com.time.util.DateUtil()
print(DateUtil.formatDateDefault(units[0].getTime()) + "-" + str(units[0].getIsAllDayTime()))

2019-01-07 15:00:00-False


In [8]:
normalizer.parse("早上六点起床")
unit = normalizer.getTimeUnit()
print(DateUtil.formatDateDefault(unit[0].getTime()) + "-" + str(unit[0].getIsAllDayTime()))

2019-01-02 06:00:00-False


In [10]:
normalizer.parse("周一开会") # 如果本周已经是周二，识别为下周周一。同理处理各级时间。（未来倾向）
unit = normalizer.getTimeUnit()
print("周一开会");
print(DateUtil.formatDateDefault(unit[0].getTime()) + "-" + str(unit[0].getIsAllDayTime()))

周一开会
2019-01-07 00:00:00-True


In [7]:
def pp(sent):
    DateUtil=gateway.jvm.com.time.util.DateUtil()
    normalizer.parse(sent)
    unit = normalizer.getTimeUnit()
    print(sent)
    print(DateUtil.formatDateDefault(unit[0].getTime()) + "-" + str(unit[0].getIsAllDayTime()))
    if len(unit)==2:
        print(DateUtil.formatDateDefault(unit[1].getTime()) + "-" + str(unit[1].getIsAllDayTime()))

pp("6月3春游") # 残缺时间的识别 （打字输入时可便捷用户）
pp("6月3日春游")

6月3春游
2019-06-01 00:00:00-True
6月3日春游
2019-06-03 00:00:00-True


In [16]:
pp("明天早上跑步") # 模糊时间范围识别（可在RangeTimeEnum中修改)
pp("本周日到下周日出差") # 多时间识别

明天早上跑步
2019-01-02 08:00:00-False
本周日到下周日出差
2019-01-06 00:00:00-True
2019-01-13 00:00:00-True


In [17]:
pp("周四下午三点到五点开会") # 多时间识别，注意第二个时间点用了第一个时间的上文

周四下午三点到五点开会
2019-01-03 15:00:00-False
2019-01-03 17:00:00-False


In [3]:
import py4j
DateUtil=gateway.jvm.com.time.util.DateUtil()
def f(object, fld):
    return py4j.java_gateway.get_field(object, fld)
def positions(words, text):
    running_offset = 0
    tokens = []
    for word in words:
        print(word)
        word_offset = text.index(word, running_offset)
        word_len = len(word)
        running_offset = word_offset + word_len
        tokens.append({"start":word_offset, "end":running_offset})
    return tokens

text="周五下午7点到8点"
normalizer=analyst.getNormalizer()
normalizer.parse(text, "2017-07-19-00-00-00")
units = normalizer.getTimeUnit()
print(text, "##", normalizer.getTarget())
words=[]
for unit in units:
    expr=f(unit, "Time_Expression")
    words.append(expr)
    print("时间文本:"+ expr +",对应时间:"+ DateUtil.formatDateDefault(unit.getTime()))
    time_full=(f(unit, "time_full"))
    print(time_full[0])
tokens= positions(words, normalizer.getTarget())
print(tokens)

周五下午7点到8点 ## 周5下午7点到8点
时间文本:周5下午7点,对应时间:2017-07-21 19:00:00
2017
时间文本:8点,对应时间:2017-07-21 20:00:00
2017
周5下午7点
8点
[{'start': 0, 'end': 6}, {'start': 7, 'end': 9}]


In [23]:
import py4j
help(py4j.java_gateway.JavaMember)

Help on class JavaMember in module py4j.java_gateway:

class JavaMember(builtins.object)
 |  Methods defined here:
 |  
 |  __call__(self, *args)
 |      Call self as a function.
 |  
 |  __init__(self, name, container, target_id, gateway_client)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  stream(self, *args)
 |      Call the method using the 'binary' protocol.
 |      
 |      :rtype: The `GatewayConnection` that the call command was sent to.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

